In [29]:
import pandas as pd
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False ## 마이나스 '-' 표시 제대로 출력
 
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [30]:
# 한글 폰트
import matplotlib.font_manager
font_list = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
print([matplotlib.font_manager.FontProperties(fname=font).get_name() for font in font_list if 'Nanum' in font][:10])
plt.rcParams["font.family"] = 'NanumSquareRound'
plt.rcParams["figure.figsize"] = (10,5)

['NanumBarunpen', 'NanumSquareOTF_ac', 'NanumGothic', 'NanumSquareOTF', 'NanumBarunpen', 'NanumBarunGothic', 'NanumSquareOTF', 'NanumSquareOTF_ac', 'NanumSquare_ac', 'NanumMyeongjo']


In [31]:
# get Data file
file_path = '../data/output/특정품목 조달 내역_2020-2022.xlsx'
data = pd.read_excel(file_path, index_col=0)
print(data.head(5))

  수요기관지역명 계약건명 수요기관구분      장비금액       계약금액    냉방용량    난방용량         날짜
0     경기도  NaN   국가기관   6034460    6034460    40.2    45.4 2020-06-08
1   서울특별시  NaN   국가기관  84395000   84395000  1013.2  1140.2 2020-10-13
2   서울특별시  NaN   국가기관     94336    2162336   249.9   281.6 2020-10-21
3   서울특별시  NaN    공기업  43490000  264110930   710.3   798.5 2020-02-26
4   서울특별시  NaN   교육기관  33878539  140333009   674.3   757.9 2020-03-24


In [32]:
# make new columns
data['용량'] = data[['냉방용량','난방용량']].max(axis=1)

# delete the row with the 0 cost
data = data[data['계약금액'] != 0]
data = data[data['장비금액'] != 0]
data = data[data['용량'] != 0]

# make new columns
data['장비금액비율'] = data['장비금액']/data['계약금액']
data = data[data['장비금액비율'] < 0.8]


In [33]:

# calculate the lower and upper bounds based on the specified quantile value
q1 = data['용량'].quantile(0.25)
q3 = data['용량'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)

In [34]:
# filter out the outlier rows
data = data[(data['용량'] >= lower_bound) & (data['용량'] <= upper_bound)]

In [35]:
def getLinearRegression(x_column: str , y_column: str , data: pd.DataFrame):
    # create a linear regression model
    reg = LinearRegression().fit(data[[x_column]], data[[y_column]])

    # use statsmodels to identify influential points
    influence = sm.OLS(data[y_column], sm.add_constant(data[x_column])).fit().get_influence()
    outliers = influence.summary_frame().loc[influence.summary_frame().cooks_d > 4/len(data)]

    # remove the influential points from the dataframe
    data_remove_outlier = data.drop(outliers.index)

    # re-fit the linear regression model without the influential points
    reg = LinearRegression().fit(data_remove_outlier[[x_column]], data_remove_outlier[[y_column]])

    # print the coefficients
    print('Intercept:', reg.intercept_)
    print('Coefficient:', reg.coef_[0])
    return reg, data_remove_outlier

In [36]:
def drawLinearRegressionPlot(regression_model: LinearRegression, x_column:str, y_column:str, data: pd.DataFrame):
    
    # plot the scatter plot and linear regression line
    fig, ax = plt.subplots()
    ax.scatter(data[x_column], data[y_column], color='blue')
    ax.plot(data[x_column], regression_model.predict(data[[x_column]]), color='red')
    ax.set_xlabel(x_column)
    ax.set_ylabel(y_column)
    ax.set_title('Scatter plot with linear regression line')

In [37]:
# create a linear regression model
reg, data_outlier = getLinearRegression(x_column='용량', y_column='계약금액', data=data)
# draw a linaer line
%matplotlib widget
drawLinearRegressionPlot(regression_model=reg, x_column='용량', y_column='계약금액', data=data_outlier)
plt.cla()
plt.show()

Intercept: [1335719.76906264]
Coefficient: [245184.37368054]


In [38]:
# create a linear regression model
reg, data_outlier = getLinearRegression(x_column='용량', y_column='장비금액비율', data=data)
# draw a linaer line
%matplotlib widget
drawLinearRegressionPlot(regression_model=reg, x_column='용량', y_column='장비금액비율', data=data_outlier)
plt.cla()
plt.show()

In [ ]:
# create boxplot
stats = data_outlier.groupby('수요기관지역명').agg({'장비금액비율': ['mean'], '계약금액':['mean']})
print(stats)